In [2]:
import numpy as np
import PIL
import PIL.Image
import torch
import json

use_gpu = torch.cuda.is_available()
device = torch.device('cuda') if use_gpu else torch.device('cpu')

/home/anna/anaconda3/envs/live/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# calculate stroke size ratio and store json record
test_direct = "data/single_strokes/single_strokes_990/sunflowers_rendered_single_stroke_0"
loss_list = []

for i in range(1,991):
    num = str(i)
    if len(num) == 1 :
        num = '00' + num
    if len(num) == 2:
        num = '0' + num
    stroke_filname = test_direct + num + ".png"
    gt = np.array(PIL.Image.open(stroke_filname))

    if gt.shape[2] == 4:
        #print("Input image includes alpha channel, simply dropout alpha channel.")
        gt = gt[:, :, :3]

    gt = (gt/255).astype(np.float32)
    gt = torch.FloatTensor(gt).permute(2, 0, 1)[None].to(device)
    gt = gt.detach().cpu().numpy()

    h, w = gt.shape[2:]
    para_bg = torch.tensor([1., 1., 1.], requires_grad=False, device=device)
    pred = para_bg.view(1, -1, 1, 1).repeat(1, 1, h, w)
    pred = pred.detach().cpu().numpy()

    map = ((pred[0] - gt[0])**2).sum(0)
    pixel_num = w*h
    zero_count = len(map[map==0.0])
    ratio = zero_count / pixel_num
    loss_list.append("{:.4f}".format(ratio))

results = {
    'loss': loss_list,
}

print(json.dumps(results,indent=4))

with open(f'{"experiment/evaluation"}/stroke_size.json', 'w') as writer:
    json.dump(results, writer, indent=4)

In [28]:
# calculate different path numbers stroke with GT and store json record
gt_direct = "data/single_strokes/single_strokes_495/sunflowers_rendered_single_stroke_"
test_direct = "log/test/sunflowers_rendered_single_stroke_"
record = {}

for i in range(1, 496) :
    num = str(i)
    if len(num) == 1 :
        num = '000' + num
    if len(num) == 2:
        num = '00' + num
    if len(num) == 3:
        num = '0' + num

    gt_filname = gt_direct +  num + ".png"
    gt = np.array(PIL.Image.open(gt_filname))

    if gt.shape[2] == 4:
        #print("Input image includes alpha channel, simply dropout alpha channel.")
        gt = gt[:, :, :3]

    gt = (gt/255).astype(np.float32)
    gt = torch.FloatTensor(gt).permute(2, 0, 1)[None].to(device)

    loss_list = []

    for j in range(1, 11) :
        
        path_num = str(j)
        stroke_filname = test_direct + num + "/demo-png/" + path_num + ".png"

        pred = np.array(PIL.Image.open(stroke_filname))

        if pred.shape[2] == 4:
            #print("Input image includes alpha channel, simply dropout alpha channel.")
            pred = pred[:, :, :3]

        pred = (pred/255).astype(np.float32)
        pred = torch.FloatTensor(pred).permute(2, 0, 1)[None].to(device)
        
        loss = ((pred-gt)**2)
        loss = loss.sum(1).mean().item()
        #print(loss)
        loss_list.append(float(loss))
    record[i] = loss_list

results = {
    'loss': record,
}

#print(json.dumps(results,indent=4))

with open(f'{"experiment/evaluation"}/results.json', 'w') as writer:
    json.dump(results, writer, indent=4)

In [ ]:
#算全部筆畫的tolerence mse path num
import json
import matplotlib.pyplot as plt
f = open('./experiment/evaluation/stroke_all_gt_mse.json')
data = json.load(f)

data = data['loss']
list = []
for item in data :
    print(item)
    mse_data = np.array(data[item])
    mse_data = mse_data.astype(np.float32)

    # Set the initial MSE value and tolerance level
    initial_mse = (mse_data[0] - mse_data[1])
    tolerance = 0.1 * initial_mse  # Set the tolerance level to 1% of the initial MSE value

    # Check for convergence
    converged = False
    for i in range(1, len(mse_data)):
        if -1*(mse_data[i] - mse_data[i-1]) < tolerance:
            list.append(i)
            print(f'Converged after {i} epochs')
            print(mse_data[i])
            converged = True
            break

    if not converged:
        list.append(len(mse_data))
        print('Did not converge')

results = {
    'stroke num': list,
}

#print(json.dumps(results,indent=4))

with open(f'{"experiment/evaluation"}/stroke_results.json', 'w') as writer:
    json.dump(results, writer, indent=4)

In [ ]:
#算單一筆畫的tolerence mse path num
import json
import matplotlib.pyplot as plt
f = open('./experiment/evaluation/stroke42_gt_mse.json')
data = json.load(f)
mse_data = np.array(data['loss'])
mse_data = mse_data.astype(np.float32)

# Set the initial MSE value and tolerance level
initial_mse = mse_data[0] - mse_data[1]
tolerance = 0.5 * initial_mse  # Set the tolerance level to 1% of the initial MSE value

# Check for convergence
converged = False
for i in range(1, len(mse_data)):
    if abs(mse_data[i] - mse_data[i-1]) < tolerance:
        list.append(i)
        print(f'Converged after {i} epochs')
        converged = True
        break

if not converged:
    print('Did not converge')

In [3]:
#根據tolerence stroke num path 合成結果
import json
import xml.etree.ElementTree as ET

svg = ET.Element('svg', width='512', height='512')
file_direct = "log/path10/sunflowers_rendered_single_stroke_"

data = json.load(open('./experiment/evaluation/stroke_results.json'))
stroke_num_data = np.array(data['stroke num'])

label = json.load( open('./experiment/evaluation/stroke_label.json') )
stroke_label_data = np.array(label['label'])

label_0 = 0
label_1 = 0 
label_2 = 0
# label_3 = 0 
# label_4 = 0
label_0_size = 0
label_1_size = 0 
label_2_size = 0
# label_3_size = 0 
# label_4_size = 0

for i in range(0,(stroke_num_data).size-1):
    if stroke_label_data[i] == 0 :
        label_0 += stroke_num_data[i]
        label_0_size += 1
    if stroke_label_data[i] == 1 :
        label_1 += stroke_num_data[i]
        label_1_size += 1
    if stroke_label_data[i] == 2 :
        label_2 += stroke_num_data[i]
        label_2_size += 1
    # if stroke_label_data[i] == 3 :
    #     label_3 += stroke_num_data[i]
    #     label_3_size += 1
    # if stroke_label_data[i] == 4 :
    #     label_4 += stroke_num_data[i]
    #     label_4_size += 1
label_0_averrage = round(label_0 / label_0_size)
label_1_averrage = round(label_1 / label_1_size)
label_2_averrage = round(label_2 / label_2_size)
# label_3_averrage = round(label_3 / label_3_size)
# label_4_averrage = round(label_4 / label_4_size)
print(label_0_averrage,label_1_averrage,label_2_averrage)
former_animateID = "a0001"
for i in range(0, 495) :
    num = str(i+1)
    if len(num) == 1 :
        num = '000' + num
    if len(num) == 2:
        num = '00' + num
    if len(num) == 3:
        num = '0' + num
    pathID = "stroke_" + num
    animateID = "a" + num
    path_num = str(4) #str(stroke_num_data[i]) # different path num
    # if stroke_label_data[i] == 0 :
    #     path_num = str(label_0_averrage)
    # if stroke_label_data[i] == 1 :
    #     path_num = str(label_1_averrage)
    # if stroke_label_data[i] == 2 :
    #     path_num = str(label_2_averrage)
    # if stroke_label_data[i] == 3 :
    #     path_num = str(label_3_averrage)
    # if stroke_label_data[i] == 4 :
    #     path_num = str(label_4_averrage)       
    stroke_filname = file_direct + num + "/output-svg/" + path_num + ".svg"
    #print(stroke_filname)
    group = ET.Element('g')
    group.set('id', pathID)
    tree1 = ET.parse(stroke_filname)
    root1 = tree1.getroot()
    for path in root1.iter('{http://www.w3.org/2000/svg}path'):
        fill_attrib = path.attrib.get('fill')
        rgb_values = fill_attrib.split(',')
        r = int(rgb_values[0][4:])
        g = int(rgb_values[1])
        b = int(rgb_values[2][:-1])
        brightness = (r + g + b) / 3
        if brightness < 240:
            group.append(path)

    animate1 = ET.Element('animate', {
        'attributeType': 'XML',
        'attributeName': 'opacity',
        'from': '0',
        'to': '0',
        'begin': '0s',
        'fill': 'freeze'
    })
    group.append(animate1)
    animate2 = ET.Element('animate', {
        'id': animateID,
        'attributeType': 'XML',
        'attributeName': 'opacity',
        'from': '0',
        'to': '1',
        'dur': '50ms',
        'begin': former_animateID + '.end+50ms',
        'fill': 'freeze'
    })
    group.append(animate2)
    svg.append(group)
    former_animateID = animateID
tree = ET.ElementTree(svg)   
tree.write('output/test/merged_4.svg')


4 4 3
